# Team 19 JHB #TeamName
---
<img src="https://github.com/Lizette95/notebook_images/blob/master/banner.png?raw=true" align="left">  


---
**Markdown Cheatsheet:**
https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet

---

### Table of Contents
---
1. [Introduction](#intro)
 * Project Description
 * Datasets and Variables
---
2. [Module Imports](#imports)
---
3. [Exploratory Data Analysis](#EDA)
 * Load Data
 * Merge Datasets
 * Data Summary Statistics
 * Missing Data
 * Numerical Features
 * Categorical Features
---
4. [Data Preprocessing](#preprocessing)
---
5. [Modelling](#modelling)
---
6. [Performance Evaluation](#evaluation)
---
7. [Model Analysis](#analysis)
---
8. [Conclusion](#conclusion)

<a id="intro"></a>
## Introduction  

### Project Description

Machine learning is a powerful branch of Artificial Intelligence, dating as far back as 1952,  developed to teach computer systems how to make decisions emanated from preexisting data. Mathematical algorithms are applied to training data which allows a machine to identify patterns and make predictions for unseen datasets.

Regression is a popular supervised learning statistical method that aims to predict the value of a dependent variable (y) based on an independent variable (x). The relationship between the variables can be linear or nonlinear.

The Zindi challenge, hosted by Sendy in partnership with insight2impact facility, is to build a regression model that will predict an accurate time of arrival for motobike deliveries from the pickup point to the destination of the package. An accurate arrival time will enhance customer communication and customer experience. In addition, the solution will help businesses reduce the cost of trade through better management of resources and planning.


The dataset provided by Sendy includes order details and rider metrics based on orders made on the Sendy platform. The challenge is to predict the estimated time of arrival for orders, from pick-up to drop-off. The training dataset is a subset of over 20,000 orders and only includes direct orders (i.e. Sendy “express” orders) with bikes in Nairobi. 

### Datasets and Variables  

**train_data:** The dataset that will be used to train our model  
**test_data:** The dataset on which we will apply our model to  
**riders:** Contains unique rider Ids, number of orders, age, rating and number of ratings

**Order details**  
* Order No: Unique number identifying the order  
* User Id: Unique number identifying the customer on a platform  
* Vehicle Type: For this competition limited to bikes, however in practice, Sendy service extends to trucks and vans  
* Platform Type: Platform used to place the order (there are 4 types)  
* Personal or Business: Customer type  

**Placement times**  
* Placement: Day of Month (i.e 1-31)  
* Placement: Weekday (Monday = 1)  
* Placement: Time - Time of day the order was placed  

**Confirmation times**  
* Confirmation: Day of Month (i.e 1-31)  
* Confirmation: Weekday (Monday = 1)  
* Confirmation: Time (Time of day the order was confirmed by a rider)  

**Arrival at Pickup times**  
* Arrival at Pickup: Day of Month (i.e 1-31)  
* Arrival at Pickup: Weekday (Monday = 1)  
* Arrival at Pickup: Time (Time of day the rider arrived at the location to pick up the order - as marked by the rider through the Sendy application)  

**Pickup times**  
* Pickup: Day of Month (i.e 1-31)  
* Pickup: Weekday (Monday = 1)  
* Pickup: Time (Time of day the rider picked up the order - as marked by the rider through the Sendy application)  

**Arrival at Destination times** (not in Test set)  
* Arrival at Delivery: Day of Month (i.e 1-31)  
* Arrival at Delivery: Weekday (Monday = 1)  
* Arrival at Delivery: Time (Time of day the rider arrived at the destination to deliver the order - as marked by the rider through the Sendy application)  
* Distance covered (KM): The distance from Pickup to Destination  
* Temperature: Temperature at the time of order placement in Degrees Celsius (measured every three hours)  
* Precipitation in Millimeters: Precipitation at the time of order placement (measured every three hours)  
* Pickup Latitude and Longitude: Latitude and longitude of pick up location  
* Destination Latitude and Longitude: Latitude and longitude of delivery location  
* Rider ID: ID of the Rider who accepted the order  
* Time from Pickup to Arrival: Time in seconds between ‘Pickup’ and ‘Arrival at Destination’    

**Rider metrics**  
* Rider ID: Unique number identifying the rider (same as in order details)  
* No of Orders: Number of Orders the rider has delivered  
* Age: Number of days since the rider delivered the first order  
* Average Rating: Average rating of the rider  
* No of Ratings: Number of ratings the rider has received. Rating an order is optional for the customer

<a id="imports"></a>
## Module Imports

In [0]:
# Ignore warnings
import warnings
warnings.simplefilter(action='ignore')

# Import modules
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Display
%matplotlib inline
sns.set_style("white")

<a id="EDA"></a>
## Exploratory Data Analysis

### Load Data

In [0]:
### Load data

### Merge Datasets

### Data Summary Statistics

### Missing Data

### Numerical Features

### Categorical Features

<a id="preprocessing"></a>
## Data Preprocessing

<a id="modelling"></a>
## Modelling

<a id="evaluation"></a>
## Performance Evaluation

<a id="analysis"></a>
## Model Analysis

<a id="conclusion"></a>
## Conclusion

Thank you and goodnight...